# Import Data

In [2]:
import pandas as pd
import cvxpy as cp
import numpy as np
#import gdown

In [6]:
df = pd.read_csv("flight_schedule.csv", index_col=0, header=1)
schedule = df.iloc[:-2]
cost = df.iloc[-2]
hours = df.iloc[-1]

data = pd.DataFrame({'year': [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
                     'generate': [8.26192344363636, 9.20460066059596, 12.0178164697778, 15.921260267805, 21.2161740066094, 31.420434564131, 38.3904519471421, 52.3307819867071, 62.9113953016839, 85.1161924282732, 104.083879757882, 132.859216030029, 170.682620580279, 220.600045153997, 276.020526299077, 346.465021938078, 440.385091980306, 530.55442135112, 635.49205101167, 705.805860788812, 831.42968828187, 962.227395409379, 1140.31094904253, 1269.52053571083, 1418.17004626655, 1591.2135122193]})
data['cum_sum'] = data['generate'].cumsum()
generate = data.generate

In [7]:
# cost
# df
data

,year,generate,cum_sum
0,1995,8.261923,8.261923
1,1996,9.204601,17.466524
2,1997,12.017816,29.484341
3,1998,15.921260,45.405601
4,1999,21.216174,66.621775
5,2000,31.420435,98.042209
6,2001,38.390452,136.432661
7,2002,52.330782,188.763443
8,2003,62.911395,251.674839
9,2004,85.116192,336.791031


In [8]:
# hours
generate

0        8.261923
1        9.204601
2       12.017816
3       15.921260
4       21.216174
5       31.420435
6       38.390452
7       52.330782
8       62.911395
9       85.116192
10     104.083880
11     132.859216
12     170.682621
13     220.600045
14     276.020526
15     346.465022
16     440.385092
17     530.554421
18     635.492051
19     705.805861
20     831.429688
21     962.227395
22    1140.310949
23    1269.520536
24    1418.170046
25    1591.213512
Name: generate, dtype: float64

In [43]:
schedule = schedule.applymap(lambda x: 1 if x >= 1 else x)

In [44]:
schedule

,1,2,3,4,5,6,7,8
Requirement (Flight),,,,,,,,
New York to Buffalo,1,0,0,1,0,0,1,0
New York to Cincinnati,0,1,0,0,1,0,0,0
New York to Chicago,0,0,1,0,0,1,0,1
Buffalo to Chicago,1,0,0,1,0,0,0,0
Chicago to Cincinnati,0,0,1,1,0,1,0,0
Cincinnati to Pittsburgh,0,1,0,1,0,1,0,0
Cincinnati to Buffalo,0,0,1,0,1,0,0,0
Buffalo to New York,0,0,1,0,1,0,1,0
Pittsburgh to New York,0,1,0,1,0,1,0,0


# Minimize Cost

Input parameters: 
* $a$: 0-1 matrix
* $c_j$: cost for each feasible sequence of flight ($j=1, 2, ..8$)
* $b_i$: requirement vector ($i=1, 2, ..., 10$)

In [45]:
a = schedule.values
c = cost.values
b = np.ones(len(a))

Decision variable: 

$y_j=1$ or $0$: Whether the flight sequence $j$ is selected

In [46]:
y = cp.Variable(len(c), boolean=True)

Constraint: ensure that at least one crew is assigned to each flight.

For example, to make sure that at least one crew is assigned to the first flight, we have constraint:
$$y_1 + y_4 + y_7 \geq 1$$

Generalize this constraint for all 10 flights, we have:

$$
\begin{pmatrix}
  1  &  0  &  0  &  1  &  0  &  0  &  1  &  0 \\
  0  &  1  &  0  &  0  &  1  &  0  &  0  &  0 \\
  0  &  0  &  1  &  0  &  0  &  1  &  0  &  1 \\
  1  &  0  &  0  &  1  &  0  &  0  &  0  &  0 \\
  0  &  0  &  1  &  1  &  0  &  1  &  0  &  0 \\
  0  &  1  &  0  &  1  &  0  &  1  &  0  &  0 \\
  0  &  0  &  1  &  0  &  1  &  0  &  0  &  0 \\
  0  &  0  &  1  &  0  &  1  &  0  &  1  &  0 \\
  0  &  1  &  0  &  1  &  0  &  1  &  0  &  0 \\
  1  &  0  &  0  &  0  &  0  &  0  &  0  &  1 
\end{pmatrix}
\begin{pmatrix}
y_1\\
y_2\\
y_3\\
y_4\\
y_5\\
y_6\\
y_7\\
y_8\\
\end{pmatrix} \geq
\begin{pmatrix}
1\\
1\\
1\\
1\\
1\\
1\\
1\\
1\\
\end{pmatrix}
$$

$y_j=0$ or $1$ $j=1,2,...,8$

In [47]:
constraints = [a @ y >= b]

Objective: minimize the total cost of assigning crews to the selected sequence of flights

$$\min \;\; z=5y_1 + 4y_2 + 4y_3 + 9y_4 + 7y_5 + 8y_6 + 3y_7 + 3y_8$$

In [48]:
obj = cp.Minimize(c @ y)

In [49]:
prob = cp.Problem(obj, constraints)

In [50]:
prob.solve(solver=cp.GLPK_MI)

SolverError: The solver GLPK_MI is not installed.

In [ ]:
print(prob.status)

None


In [ ]:
print(y.value)

[1. 1. 1. 0. 0. 0. 0. 0.]


# Hours

In [ ]:
h = hours.values
h

array([ 396,  352, 1022,  847,  687,  531,  236,  179], dtype=int64)

Constraints:
* There must be at least one airplane per flight
$$a \cdot y\geq b$$

* The total hours of flights must be less than or equal to 1700
$$h \cdot y \leq 1700$$

In [ ]:
constraints = [a @ y >= b, h @ y <= 1700]

In [ ]:
prob = cp.Problem(obj, constraints)

In [ ]:
prob.solve(solver=cp.GLPK_MI)

20.0

In [ ]:
print(y.value)

[1. 0. 0. 0. 1. 1. 0. 0.]


$$396 + 352 + 1022 = 1770$$
$$396 + 687 + 531 = 1614$$